In [2]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/07/05 09:33:40 WARN Utils: Your hostname, DESKTOP-EJLBN3A resolves to a loopback address: 127.0.1.1; using 172.18.30.182 instead (on interface eth0)
22/07/05 09:33:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/05 09:33:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
from pyspark.sql import SparkSession
from datetime import date

#create API connection and authenticate
api = KaggleApi()
api.authenticate()

#download dataset 
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')

#create spark session
# spark = SparkSession.builder.getOrCreate()

#create spark df
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def date_check():
    today = date.today()
    return str(today)

def column_grab():
    try:
        return daily_df.select(daily_df[date_check()]).show()
    except:
        pass

column_grab()

In [41]:
# DATA CLEANING
from pyspark.sql.types import IntegerType

daily_df = spark.read.csv("../data/Daily_Country_Wise_Confirmed_Cases.csv", header=True)
worldwide_df = spark.read.csv("../data/Monkey_Pox_Cases_Worldwide.csv", header=True)
timeline_df = spark.read.csv("../data/Worldwide_Case_Detection_Timeline.csv", header=True)
# daily_df.show(10)
# daily_df.describe()
# len(daily_df.columns)
worldwide_df.columns
worldwide_df.show(2)
worldwide_df = worldwide_df.withColumns({"Confirmed_Cases": worldwide_df["Confirmed_Cases"].cast(IntegerType()), "Suspected_Cases": worldwide_df["Suspected_Cases"].cast(IntegerType()), "Hospitalized": worldwide_df["Hospitalized"].cast(IntegerType()), "Travel_History_Yes": worldwide_df["Travel_History_Yes"].cast(IntegerType()), "Travel_History_No": worldwide_df["Travel_History_No"].cast(IntegerType())})
worldwide_df.show(2)
# timeline_df = timeline_df.fillna({"Age": "unknown", "Gender": "unknown", "Symptoms": "unknown", "Hospitalised (Y/N/NA)": "NA", "Isolated (Y/N/NA)": "NA", "Travel_History (Y/N/NA)": "NA"})
# timeline_df.show(30)

+--------+---------------+---------------+------------+------------------+-----------------+
| Country|Confirmed_Cases|Suspected_Cases|Hospitalized|Travel_History_Yes|Travel_History_No|
+--------+---------------+---------------+------------+------------------+-----------------+
| England|         1035.0|            0.0|         5.0|               2.0|              7.0|
|Portugal|          402.0|            0.0|         0.0|               0.0|             34.0|
+--------+---------------+---------------+------------+------------------+-----------------+
only showing top 2 rows

+--------+---------------+---------------+------------+------------------+-----------------+
| Country|Confirmed_Cases|Suspected_Cases|Hospitalized|Travel_History_Yes|Travel_History_No|
+--------+---------------+---------------+------------+------------------+-----------------+
| England|           1035|              0|           5|                 2|                7|
|Portugal|            402|              0|   